In [1]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import pandas as pd
import numpy as np
import gc
import os
import time

### Get the data from BQ bucket

In [1]:
!gsutil -m cp gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_* Data/

Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000000...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000001...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000003...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000002...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000004...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000005...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000006...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000008...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000009...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000007...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000010...
Copying gs://wildfire-yz/consolidated2/consolidated_by_cell_day_4_000000000012...
Copying gs://wil

In [15]:
data_dir = '/home/yulia/Documents/MIDS/W210/Model/Data/'
# data_dir = '/tf/notebooks/W210/Model/Data/'

In [17]:
allfiles = os.listdir(data_dir)
allfiles

['consolidated_by_cell_day_4_000000000012',
 'consolidated_by_cell_day_4_000000000007',
 'consolidated_by_cell_day_4_000000000008',
 'consolidated_by_cell_day_4_000000000015',
 'consolidated_by_cell_day_4_000000000014',
 'consolidated_by_cell_day_4_000000000003',
 'consolidated_by_cell_day_4_000000000011',
 'consolidated_by_cell_day_4_000000000002',
 'consolidated_by_cell_day_4_000000000005',
 'consolidated_by_cell_day_4_000000000000',
 'consolidated_by_cell_day_4_000000000004',
 'consolidated_by_cell_day_4_000000000001',
 'consolidated_by_cell_day_4_000000000016',
 'consolidated_by_cell_day_4_000000000013',
 'consolidated_by_cell_day_4_000000000006',
 'consolidated_by_cell_day_4_000000000010',
 'consolidated_by_cell_day_4_000000000009']

In [18]:
keep_wf_cols = [
    'S2_Cells_ID',
    'WF_FIRE_DATE',
    'WF_WildFire',
    'WF_WildFire_COUNT_1YR_AGO',
    'WF_WildFire_COUNT_2YR_AGO',
    'WF_WildFire_COUNT_3YR_AGO',
    'WF_WildFire_COUNT_4YR_AGO',
    'WF_WildFire_COUNT_5YR_AGO'
]

new_wf_cols = [
    's2_cell_id',
    'measure_date',
    'wf_wildfire_ext',         
    'wf_count_1yr_ago',    
    'wf_count_2yr_ago',    
    'wf_count_3yr_ago',    
    'wf_count_4yr_ago',    
    'wf_count_5yr_ago'    
]

wf_hist_cols = [
    'wf_count_1yr_ago',    
    'wf_count_2yr_ago',    
    'wf_count_3yr_ago',    
    'wf_count_4yr_ago',    
    'wf_count_5yr_ago'    
]

In [10]:
wf_ext_df = pd.read_csv('../Data/Processed/WildFire_S2Cells_Extended.csv')
wf_ext_df = wf_ext_df[keep_wf_cols]
wf_ext_df.columns = new_wf_cols
wf_ext_df.shape

(86689, 8)

In [13]:
wf_ext_df.wf_wildfire_ext.sum()

86689

In [11]:
wf_ext_df.head()

,s2_cell_id,measure_date,wf_wildfire_ext,wf_count_1yr_ago,wf_count_2yr_ago,wf_count_3yr_ago,wf_count_4yr_ago,wf_count_5yr_ago
0,54c942c,2017-07-24,1,0,0,0,0,0
1,54c942c,2017-07-25,1,0,0,0,0,0
2,54c942c,2017-07-26,1,0,0,0,0,0
3,54c942c,2017-07-27,1,0,0,0,0,0
4,54c942c,2017-07-28,1,0,0,0,0,0


### Get 10% sample of non-wildfire cells

In [20]:
def sample_data(filepath, sample=0.1):
    temp = pd.read_csv(filepath)
    print(f"Observations in raw: {temp.shape[0]}")
    temp.drop(columns=wf_hist_cols, inplace=True)
    temp = temp.merge(wf_ext_df, on=['s2_cell_id', 'measure_date'], how='left')
    temp_wf = temp[~temp.wf_wildfire_ext.isna()]
    print(f"WF Observations in raw: {temp_wf.shape[0]}")
    temp_nwf = temp[temp.wf_wildfire_ext.isna()].sample(frac=sample, random_state=1)
    print(f"NWF Observations sampled: {temp_nwf.shape[0]}")
    temp = temp_wf.append(temp_nwf)
    print(f"Observations in processed: {temp.shape[0]}")
    return temp

In [21]:
final_df = pd.DataFrame()
for file in allfiles:
    if file[0] == 'c':
        print(f"Processing File {file}")
        path = data_dir + file
        final_df = final_df.append(sample_data(path))
        print(f"Observations in the dataset so far: {final_df.shape[0]}")
final_df.reset_index(drop=True,inplace=True)

Processing File consolidated_by_cell_day_4_000000000012


/home/yulia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (97,98,99,100,101) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Observations in raw: 686543
WF Observations in raw: 5218
NWF Observations sampled: 68132
Observations in processed: 73350
Observations in the dataset so far: 73350
Processing File consolidated_by_cell_day_4_000000000007
Observations in raw: 686355
WF Observations in raw: 5060
NWF Observations sampled: 68130
Observations in processed: 73190
Observations in the dataset so far: 146540
Processing File consolidated_by_cell_day_4_000000000008
Observations in raw: 684324
WF Observations in raw: 5069
NWF Observations sampled: 67926
Observations in processed: 72995
Observations in the dataset so far: 219535
Processing File consolidated_by_cell_day_4_000000000015
Observations in raw: 685772
WF Observations in raw: 5078
NWF Observations sampled: 68069
Observations in processed: 73147
Observations in the dataset so far: 292682
Processing File consolidated_by_cell_day_4_000000000014
Observations in raw: 685737
WF Observations in raw: 5037
NWF Observations sampled: 68070
Observations in processed: 7

In [22]:
gc.collect()

202

In [23]:
final_df.shape

(1244496, 166)

In [24]:
final_df.s2_cell_id.nunique()

10643

In [26]:
final_df.wf_wildfire.sum(), final_df.wf_wildfire_ext.sum()

(3673.0, 86689.0)

In [27]:
final_df.wf_wildfire_ext.sum() / final_df.shape[0]

0.06965791774340778

### Checking features

In [33]:
final_df[final_df.fuel_percent_l1=='backfill'].shape

(12175, 166)

In [ ]:
final_df[final_df.fuel_percent=='backfill'].shape

In [30]:
fuel_mean = 78.74 # Calculated in BigQuery
final_df.fuel_percent.replace('backfill',str(fuel_mean),inplace = True)
final_df.fuel_percent_l1.replace('backfill',str(fuel_mean),inplace = True)
final_df['fuel_percent'] = pd.to_numeric(final_df.fuel_percent)
final_df['fuel_percent_l1'] = pd.to_numeric(final_df.fuel_percent_l1)

In [35]:
final_df[[col for col in final_df if col.startswith('fuel_')]].dtypes

fuel_uid              object
fuel_percent         float64
fuel_percent_ma7     float64
fuel_percent_ma30    float64
fuel_percent_l1      float64
dtype: object

In [36]:
final_df[[col for col in final_df if col.startswith('fuel_')]].isna().sum()

fuel_uid                0
fuel_percent            0
fuel_percent_ma7        0
fuel_percent_ma30       0
fuel_percent_l1      1070
dtype: int64

In [37]:
final_df[[col for col in final_df if col.startswith('tl_')]].dtypes

tl_object_id                      float64
tl_kv_sort_sum                    float64
tl_kv_sort_mean                   float64
tl_kv_sort_max                    float64
tl_owner                          float64
tl_circuit                        float64
tl_length_mil_sum                 float64
tl_length_mil_mean                float64
tl_length_mil_max                 float64
tl_owner_amp                      float64
tl_owner_anza                     float64
tl_owner_apud                     float64
tl_owner_blythe_energy            float64
tl_owner_bpa                      float64
tl_owner_bves                     float64
tl_owner_calpeco                  float64
tl_owner_ccsf                     float64
tl_owner_esj                      float64
tl_owner_iid                      float64
tl_owner_ipa                      float64
tl_owner_kmpud                    float64
tl_owner_ladwp                    float64
tl_owner_lmud                     float64
tl_owner_mid                      

In [38]:
final_df[[col for col in final_df if col.startswith('tl_')]].isna().sum()

tl_object_id                      764748
tl_kv_sort_sum                    764748
tl_kv_sort_mean                   764748
tl_kv_sort_max                    764748
tl_owner                          764748
tl_circuit                        764748
tl_length_mil_sum                 764748
tl_length_mil_mean                764857
tl_length_mil_max                 764857
tl_owner_amp                      764748
tl_owner_anza                     764748
tl_owner_apud                     764748
tl_owner_blythe_energy            764748
tl_owner_bpa                      764748
tl_owner_bves                     764748
tl_owner_calpeco                  764748
tl_owner_ccsf                     764748
tl_owner_esj                      764748
tl_owner_iid                      764748
tl_owner_ipa                      764748
tl_owner_kmpud                    764748
tl_owner_ladwp                    764748
tl_owner_lmud                     764748
tl_owner_mid                      764748
tl_owner_mwd    

In [39]:
final_df[[col for col in final_df if col.startswith('wf_')]].dtypes

wf_wildfire         float64
wf_gis_acres        float64
wf_fire_dur         float64
wf_agency_bia       float64
wf_agency_blm       float64
wf_agency_coco      float64
wf_agency_cdf       float64
wf_agency_dod       float64
wf_agency_lra       float64
wf_agency_nps       float64
wf_agency_usf       float64
wf_cause_1          float64
wf_cause_2          float64
wf_cause_3          float64
wf_cause_4          float64
wf_cause_5          float64
wf_cause_6          float64
wf_cause_7          float64
wf_cause_8          float64
wf_cause_9          float64
wf_cause_10         float64
wf_cause_11         float64
wf_cause_14         float64
wf_cause_15         float64
wf_cause_16         float64
wf_cause_18         float64
wf_c_method_1       float64
wf_c_method_2       float64
wf_c_method_3       float64
wf_c_method_4       float64
wf_c_method_5       float64
wf_c_method_6       float64
wf_c_method_7       float64
wf_c_method_8       float64
wf_cum_area         float64
wf_wildfire_ext     

In [41]:
final_df[[col for col in final_df if col.startswith('sat_')]].dtypes

sat_faparval_min         object
sat_faparval_max         object
sat_faparval_mean        object
sat_faparval_median      object
sat_faparval_std         object
sat_faparval_size         int64
sat_faparval_count        int64
sat_faparval              int64
sat_faparmask_min          bool
sat_faparmask_max          bool
sat_faparmask_mean      float64
sat_faparmask_median     object
sat_faparmask_std       float64
sat_faparmask_size        int64
dtype: object

In [51]:
print((final_df.sat_faparval_min=='--').sum())
print((final_df.sat_faparval_max=='--').sum())
print((final_df.sat_faparval_mean=='--').sum())
print((final_df.sat_faparval_median=='--').sum())
print((final_df.sat_faparval_std=='--').sum())

1470
1470
1470
1470
1470


In [56]:
final_df.sat_faparmask_median.value_counts()

0.0      605521
False    597150
1.0       34148
True       7506
0.5         171
Name: sat_faparmask_median, dtype: int64

In [52]:
final_df['sat_faparval_undefined'] = (final_df.sat_faparval_min=='--').astype(int)
final_df['sat_faparval_undefined'].value_counts()

0    1243026
1       1470
Name: sat_faparval_undefined, dtype: int64

In [53]:
final_df['sat_faparval_min'] = pd.to_numeric(final_df.sat_faparval_min, errors='coerce')
final_df['sat_faparval_max'] = pd.to_numeric(final_df.sat_faparval_max, errors='coerce')
final_df['sat_faparval_mean'] = pd.to_numeric(final_df.sat_faparval_mean, errors='coerce')
final_df['sat_faparval_median'] = pd.to_numeric(final_df.sat_faparval_median, errors='coerce')
final_df['sat_faparval_std'] = pd.to_numeric(final_df.sat_faparval_std, errors='coerce')

In [58]:
final_df['sat_faparval_min'] = final_df.sat_faparval_min.fillna(0)
final_df['sat_faparval_max'] = final_df.sat_faparval_max.fillna(0)
final_df['sat_faparval_mean'] = final_df.sat_faparval_mean.fillna(0)
final_df['sat_faparval_median'] = final_df.sat_faparval_median.fillna(0)
final_df['sat_faparval_std'] = final_df.sat_faparval_std.fillna(0)

In [59]:
final_df[[col for col in final_df if col.startswith('sat_')]].isna().sum()

sat_faparval_min          0
sat_faparval_max          0
sat_faparval_mean         0
sat_faparval_median       0
sat_faparval_std          0
sat_faparval_size         0
sat_faparval_count        0
sat_faparval              0
sat_faparmask_min         0
sat_faparmask_max         0
sat_faparmask_mean        0
sat_faparmask_median      0
sat_faparmask_std         0
sat_faparmask_size        0
sat_faparval_undefined    0
dtype: int64

In [61]:
final_df[[col for col in final_df if col.startswith('sat_')]].describe().round(2)

,sat_faparval_min,sat_faparval_max,sat_faparval_mean,sat_faparval_median,sat_faparval_std,sat_faparval_size,sat_faparval_count,sat_faparval,sat_faparmask_mean,sat_faparmask_std,sat_faparmask_size,sat_faparval_undefined
count,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00,1244496.00
mean,0.16,0.63,0.38,0.38,0.09,569.76,569.76,106.06,0.04,0.06,569.76,0.00
std,0.15,0.21,0.20,0.21,0.05,735.53,735.53,96.49,0.15,0.13,735.53,0.03
min,0.00,0.00,0.00,0.00,0.00,182.00,182.00,1.00,0.00,0.00,182.00,0.00
25%,0.04,0.50,0.23,0.21,0.06,199.00,199.00,63.00,0.00,0.00,199.00,0.00
50%,0.11,0.67,0.38,0.39,0.09,211.00,211.00,85.00,0.00,0.00,211.00,0.00
75%,0.25,0.80,0.54,0.54,0.12,788.00,788.00,116.00,0.00,0.03,788.00,0.00
max,0.85,0.94,0.92,0.94,0.38,3647.00,3647.00,1811.00,1.00,0.50,3647.00,1.00


In [65]:
weather = [
 'wea_air_temp_mean_l1',
 'wea_precip_accum_max_l1',
 'relative_humidity_mean_l1',
 'wea_air_temp_mean_ma7',
 'wea_precip_accum_max_ma7',
 'relative_humidity_mean_ma7',
 'wea_air_temp_mean_ma30',
 'wea_precip_accum_max_ma30',
 'relative_humidity_mean_ma30',
 'wea_air_temp_max',
 'wea_air_temp_mean',
 'wea_precip_accum_max',
 'relative_humidity_max',
 'relative_humidity_min',
 'relative_humidity_mean',
 'wea_wind_speed_max',
 'wea_wind_speed_min',
 'wea_wind_speed_mean',
 'wind_gust_max',
]

In [66]:
final_df[weather].dtypes

wea_air_temp_mean_l1           float64
wea_precip_accum_max_l1        float64
relative_humidity_mean_l1      float64
wea_air_temp_mean_ma7          float64
wea_precip_accum_max_ma7       float64
relative_humidity_mean_ma7     float64
wea_air_temp_mean_ma30         float64
wea_precip_accum_max_ma30      float64
relative_humidity_mean_ma30    float64
wea_air_temp_max               float64
wea_air_temp_mean              float64
wea_precip_accum_max           float64
relative_humidity_max          float64
relative_humidity_min          float64
relative_humidity_mean         float64
wea_wind_speed_max             float64
wea_wind_speed_min             float64
wea_wind_speed_mean            float64
wind_gust_max                  float64
dtype: object

In [67]:
final_df[weather].isna().sum()

wea_air_temp_mean_l1           1070
wea_precip_accum_max_l1        1070
relative_humidity_mean_l1      1070
wea_air_temp_mean_ma7             0
wea_precip_accum_max_ma7          0
relative_humidity_mean_ma7        0
wea_air_temp_mean_ma30            0
wea_precip_accum_max_ma30         0
relative_humidity_mean_ma30       0
wea_air_temp_max                  0
wea_air_temp_mean                 0
wea_precip_accum_max              0
relative_humidity_max             0
relative_humidity_min             0
relative_humidity_mean            0
wea_wind_speed_max                0
wea_wind_speed_min                0
wea_wind_speed_mean               0
wind_gust_max                     0
dtype: int64

In [68]:
gc.collect()

227

#### Replace missing values for the lag variables

In [74]:
final_df[final_df.measure_date=='2016-01-01'].shape

(1070, 167)

In [78]:
final_df = final_df[final_df.measure_date!='2016-01-01']
final_df.shape

(1243426, 167)

In [91]:
gc.collect()

162

In [86]:
final_df.fz_state_zone.head()

0    CA270
1    CA270
2    CA278
3    CA278
4    CA278
Name: fz_state_zone, dtype: object

In [89]:
control_vars = [
 's2_cell_id',
 'measure_date',
 'fz_grp_high',
 'fz_grp_low',
 'fz_grp_med',
 'fz_state_zone',
 'wf_wildfire',
 'wf_wildfire_ext',
]

tl_features = [
    'tl_object_id',
    'tl_kv_sort_sum',
    'tl_s2_cells_lvl16_cnt',
    'tl_s2_cells_lvl16_cnt_coverage'
]

wf_features = [
    'wf_count_1yr_ago',
    'wf_count_2yr_ago',
    'wf_count_3yr_ago',
    'wf_count_4yr_ago',
    'wf_count_5yr_ago'
]

sat_features = [
 'sat_faparval_min',
 'sat_faparval_max',
 'sat_faparval_mean',
 'sat_faparval_median',
 'sat_faparval_std',
 'sat_faparval_size',
 'sat_faparval_count',
 'sat_faparval',
 'sat_faparmask_min',
 'sat_faparmask_max',
 'sat_faparmask_mean',
#  'sat_faparmask_median',
 'sat_faparmask_std',
 'sat_faparmask_size',
 'sat_faparval_undefined'
]

wea_features = [
 'wea_air_temp_max',
 'wea_air_temp_mean',
 'wea_precip_accum_max',
 'relative_humidity_max',
 'relative_humidity_min',
 'relative_humidity_mean',
 'wea_wind_speed_max',
 'wea_wind_speed_min',
 'wea_wind_speed_mean',
 'wind_gust_max',
 'wea_air_temp_mean_ma7',
 'wea_precip_accum_max_ma7',
 'relative_humidity_mean_ma7',
 'wea_air_temp_mean_l1',
 'wea_precip_accum_max_l1',
 'relative_humidity_mean_l1',
 'wea_air_temp_mean_ma30',
 'wea_precip_accum_max_ma30',
 'relative_humidity_mean_ma30'
]

fuel_features = [
 'fuel_percent',
 'fuel_percent_ma7',
 'fuel_percent_ma30',
 'fuel_percent_l1',
]

In [90]:
final_df[tl_features+sat_features+wea_features+fuel_features].mean()

tl_object_id                        4.720792
tl_kv_sort_sum                    721.809149
tl_s2_cells_lvl16_cnt             166.028750
tl_s2_cells_lvl16_cnt_coverage      0.057191
sat_faparval_min                    0.157561
sat_faparval_max                    0.632617
sat_faparval_mean                   0.384947
sat_faparval_median                 0.383896
sat_faparval_std                    0.095002
sat_faparval_size                 569.752300
sat_faparval_count                569.752300
sat_faparval                      106.056871
sat_faparmask_min                   0.001177
sat_faparmask_max                   0.257098
sat_faparmask_mean                  0.043190
sat_faparmask_std                   0.056967
sat_faparmask_size                569.752300
sat_faparval_undefined              0.001177
wea_air_temp_max                   21.225725
wea_air_temp_mean                  13.924206
wea_precip_accum_max              450.377858
relative_humidity_max              75.055724
relative_h

In [92]:
keep_cols = control_vars+wf_features+tl_features+sat_features+wea_features+fuel_features
final_df = final_df[keep_cols]
final_df.shape

(1243426, 54)

In [95]:
final_df.fillna(0, inplace=True)

/home/yulia/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [96]:
final_df.isna().sum()

s2_cell_id                        0
measure_date                      0
fz_grp_high                       0
fz_grp_low                        0
fz_grp_med                        0
fz_state_zone                     0
wf_wildfire                       0
wf_wildfire_ext                   0
wf_count_1yr_ago                  0
wf_count_2yr_ago                  0
wf_count_3yr_ago                  0
wf_count_4yr_ago                  0
wf_count_5yr_ago                  0
tl_object_id                      0
tl_kv_sort_sum                    0
tl_s2_cells_lvl16_cnt             0
tl_s2_cells_lvl16_cnt_coverage    0
sat_faparval_min                  0
sat_faparval_max                  0
sat_faparval_mean                 0
sat_faparval_median               0
sat_faparval_std                  0
sat_faparval_size                 0
sat_faparval_count                0
sat_faparval                      0
sat_faparmask_min                 0
sat_faparmask_max                 0
sat_faparmask_mean          

In [97]:
final_df.isna().sum().sum()

0

In [98]:
final_df.shape

(1243426, 54)

In [100]:
fz_dummies = pd.get_dummies(final_df.fz_state_zone)
fz_dummies.shape

(1243426, 105)

In [103]:
final_df.wea_precip_accum_max.describe().round(3)

count    1243426.000
mean         450.378
std         4051.479
min          -95.250
25%           38.100
50%          230.632
75%          586.740
max      1849780.400
Name: wea_precip_accum_max, dtype: float64

In [105]:
final_df[final_df.wea_precip_accum_max<0].shape

(6, 54)

In [107]:
final_df.relative_humidity_min.describe().round(3)

count    1243426.000
mean          31.354
std          448.508
min      -127888.000
25%           16.060
50%           27.000
75%           46.000
max          250.000
Name: relative_humidity_min, dtype: float64

In [108]:
final_df[final_df.relative_humidity_min<0].shape

(49, 54)

In [109]:
final_df[final_df.relative_humidity_min<0].head()

,s2_cell_id,measure_date,fz_grp_high,fz_grp_low,fz_grp_med,fz_state_zone,wf_wildfire,wf_wildfire_ext,wf_count_1yr_ago,wf_count_2yr_ago,...,wea_air_temp_mean_l1,wea_precip_accum_max_l1,relative_humidity_mean_l1,wea_air_temp_mean_ma30,wea_precip_accum_max_ma30,relative_humidity_mean_ma30,fuel_percent,fuel_percent_ma7,fuel_percent_ma30,fuel_percent_l1
29027,54cd9fc,2018-12-19,1,0,0,CA213,0.0,0.0,0.0,0.0,...,6.661757,2413.000,88.767123,-25.054228,572.592200,30.737721,70.0,74.857143,67.600000,72.0
36188,8085fc,2018-03-14,0,1,0,CA006,0.0,0.0,0.0,0.0,...,-4567.780000,7.366,-100.000000,-4567.780000,6.680200,-100.000000,66.0,82.571429,80.433333,66.0
83652,80971c4,2018-04-07,1,0,0,CA592,0.0,0.0,0.0,0.0,...,3.196250,5.588,76.239583,-0.154079,197.662800,61.624237,156.0,91.857143,84.966667,89.0
93274,809c95,2018-12-11,0,0,1,CA268,0.0,0.0,0.0,0.0,...,6.198246,1062.482,85.100000,-254.615920,238.252000,-301.075668,74.0,72.285714,59.733333,74.0
106716,80bfe0c,2018-10-20,1,0,0,CA594,0.0,0.0,0.0,0.0,...,2.507396,36.576,44.833333,5.432586,175.683333,-878.964206,87.0,61.857143,52.833333,59.0


In [138]:
final_df.relative_humidity_mean.describe().round()

count    1243426.0
mean          56.0
std           42.0
min       -28062.0
25%           37.0
50%           55.0
75%           76.0
max         1624.0
Name: relative_humidity_mean, dtype: float64

In [139]:
final_df[final_df.relative_humidity_mean>70].wea_precip_accum_max.describe().round(3)

count     398623.000
mean         479.587
std         3211.475
min          -95.250
25%           85.598
50%          266.192
75%          625.094
max      1404848.600
Name: wea_precip_accum_max, dtype: float64

In [140]:
final_df[final_df.relative_humidity_mean<90].wea_precip_accum_max.describe().round(3)

count    1142472.000
mean         441.931
std         4010.565
min          -95.250
25%           33.274
50%          221.488
75%          571.754
max      1849780.400
Name: wea_precip_accum_max, dtype: float64

In [141]:
final_df[final_df.relative_humidity_mean<70].wea_precip_accum_max.describe().round(3)

count     844371.000
mean         436.506
std         4393.435
min          -95.250
25%           21.590
50%          210.566
75%          565.404
max      1849780.400
Name: wea_precip_accum_max, dtype: float64

In [142]:
final_df[final_df.relative_humidity_mean<50].wea_precip_accum_max.describe().round(3)

count     533010.000
mean         414.511
std         4385.070
min            0.000
25%           13.208
50%          167.386
75%          523.240
max      1849780.400
Name: wea_precip_accum_max, dtype: float64

In [143]:
final_df['wea_precip_accum_max_cut90'] = np.where(final_df['relative_humidity_mean']<90, 0, final_df.wea_precip_accum_max)
final_df['wea_precip_accum_max_cut70'] = np.where(final_df['relative_humidity_mean']<70, 0, final_df.wea_precip_accum_max)
final_df['wea_precip_accum_max_cut50'] = np.where(final_df['relative_humidity_mean']<50, 0, final_df.wea_precip_accum_max)

final_df['wea_precip_accum_max_ma7_cut90'] = np.where(final_df['relative_humidity_mean_ma7']<90, 0, final_df.wea_precip_accum_max_ma7)
final_df['wea_precip_accum_max_ma7_cut70'] = np.where(final_df['relative_humidity_mean_ma7']<70, 0, final_df.wea_precip_accum_max_ma7)
final_df['wea_precip_accum_max_ma7_cut50'] = np.where(final_df['relative_humidity_mean_ma7']<50, 0, final_df.wea_precip_accum_max_ma7)

final_df['wea_precip_accum_max_ma30_cut90'] = np.where(final_df['relative_humidity_mean_ma30']<90, 0, final_df.wea_precip_accum_max_ma30)
final_df['wea_precip_accum_max_ma30_cut70'] = np.where(final_df['relative_humidity_mean_ma30']<70, 0, final_df.wea_precip_accum_max_ma30)
final_df['wea_precip_accum_max_ma30_cut50'] = np.where(final_df['relative_humidity_mean_ma30']<50, 0, final_df.wea_precip_accum_max_ma30)

final_df['wea_precip_accum_max_l1_cut90'] = np.where(final_df['relative_humidity_mean_l1']<90, 0, final_df.wea_precip_accum_max_l1)
final_df['wea_precip_accum_max_l1_cut70'] = np.where(final_df['relative_humidity_mean_l1']<70, 0, final_df.wea_precip_accum_max_l1)
final_df['wea_precip_accum_max_l1_cut50'] = np.where(final_df['relative_humidity_mean_l1']<50, 0, final_df.wea_precip_accum_max_l1)

In [144]:
final_df.to_csv('./consolidated_4_10pct_sample_new.csv', index=False)

In [146]:
final_df.isna().sum().sum()

0